In [3]:
import telebot
import io
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import seaborn as sns

token='YOUR_TOKEN'
bot=telebot.TeleBot(token)

# /start
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Привет! Я бот - аналитик данных. Чтобы посомотреть, что я умею, введите /help")

# /help
@bot.message_handler(commands=['help'])
def start_message(message):
  bot.send_message(message.chat.id, '''Я умею выполнять следующие функции:
/csv_info - вы загружаете CSV-файл и получаете информацию о количестве строк и колонок датафрейма (shape);
/csv_cut - вы загружаете CSV-файл, я убираю в нем все строки, кроме первой, и возвращаю вам отредактированный файл;
/csv_distr - вы загружаете CSV-файл и задаете название колонки, а я строю распределение данных в этой колонке (displot)''')


# /csv_info
# Обработка команды /csv_info
@bot.message_handler(commands=['csv_info'])
def handle_csv_info(message):
    bot.send_message(message.chat.id, "Загрузите csv-файл")
    bot.register_next_step_handler(message, handle_info)
# Обработка загруженного CSV-файла
def handle_info(message):
    try:
        if message.document.mime_type == 'text/csv':
            file_id = message.document.file_id
            file_info = bot.get_file(file_id)
            downloaded_file = bot.download_file(file_info.file_path)
            df = pd.read_csv(io.StringIO(downloaded_file.decode()))

            # Отправляем информацию о размере датафрейма
            bot.send_message(message.chat.id, "Размер датафрейма: {} строк и {} колонок".format(df.shape[0], df.shape[1]))
            # Отправляем информацию о колонках и типах данных
            bot.send_message(message.chat.id, "Название колонок и типы данных:\n\n{}".format(df.dtypes))

        else:
            bot.send_message(message.chat.id, "Принимаются только CSV файлы")
    except Exception as e:
        bot.send_message(message.chat.id, "Ошибка. Возможно вы не загрузили CSV файл на предыдущем шаге. Повторите нужную команду.")

# Канву функции ниже можно использовать для создания любых функций, обрабатывающих весь датафрейм одной задачей
# /csv_cut
# Обработка команды /csv_cut
@bot.message_handler(commands=['csv_cut'])
def handle_csv_info(message):
    bot.send_message(message.chat.id, "Загрузите csv-файл")
    bot.register_next_step_handler(message, handle_docs_photo)
# Обработка загруженного CSV-файла
def handle_docs_photo(message):
    try:
        if message.document.mime_type == 'text/csv':
            file_id = message.document.file_id
            file_info = bot.get_file(file_id)
            downloaded_file = bot.download_file(file_info.file_path)
            df = pd.read_csv(io.StringIO(downloaded_file.decode()))
            # Оставляем только первую строку в DataFrame
            df = df.iloc[:1,:]
            # Отправляем обработанный файл в чат
            processed_file = io.StringIO()
            df.to_csv(processed_file, index=False)
            processed_file.seek(0)
            bot.send_document(message.chat.id, processed_file, visible_file_name = 'processed_file.csv')
            bot.send_message(message.chat.id, "Обработанный файл отправлен!")
        else:
            bot.send_message(message.chat.id, "Принимаются только CSV файлы")
    except Exception as e:
        bot.send_message(message.chat.id, "Ошибка. Возможно вы не загрузили CSV файл на предыдущем шаге. Повторите нужную команду.")

# Канву функции ниже можно использовать для создания любых функций, обрабатывающих указанные столбцы датафрейма одной задачей
# /csv_distr
# Обработка команды /csv_distr
@bot.message_handler(commands=['csv_distr'])
def handle_csv_distr(message):
    bot.send_message(message.chat.id, "Загрузите csv-файл")
    bot.register_next_step_handler(message, handle_distr)
# Обработка загруженного CSV-файла
def handle_distr(message):
    try:
        if message.document.mime_type == 'text/csv':
            file_id = message.document.file_id
            file_info = bot.get_file(file_id)
            downloaded_file = bot.download_file(file_info.file_path)
            global df
            df = pd.read_csv(io.StringIO(downloaded_file.decode()))
            bot.send_message(message.chat.id, "Введите название колонки для распределения: ")
            bot.register_next_step_handler(message, handle_distr_column)
        else:
            bot.send_message(message.chat.id, "Принимаются только CSV файлы")
    except Exception as e:
        bot.send_message(message.chat.id, "Ошибка. Возможно вы не загрузили CSV файл на предыдущем шаге. Повторите нужную команду.")
        #bot.register_next_step_handler(message, handle_distr_column)
# Построение распределения по колонке
def handle_distr_column(message):
    try:
        column_name = message.text
        column = df[column_name]
        #plt.figure(figsize=(20, 15))
        #plt.xticks(rotation=45)
        sns.displot(column)
        plt.xticks(rotation=45)
        plt.savefig("distribution.png")
        with open("distribution.png", "rb") as photo:
            bot.send_photo(message.chat.id, photo)
        os.remove("distribution.png")
    except KeyError:
        bot.send_message(message.chat.id, "Колонка не найдена либо неподходящий тип данных")
        plt.clf()

 

# постоянный мониториг чата на команды
bot.polling(none_stop=True)